In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import sys
Image.MAX_IMAGE_PIXELS = 999999999

In [2]:
src = r'\\motherserverdw\Kyu_Sync\server for undergrads\JongHa'
csvs = [_ for _ in os.listdir(src) if _.endswith('csv')]
# csvs = [_ for _ in csvs if '017' in _]
tiles = [os.path.splitext(_.replace('parameters','tile'))[0] for _ in csvs]
images = [os.path.join(_,'image') for _ in tiles]
masks = [os.path.join(_,'mask') for _ in tiles]


In [3]:
for csv_nm,tile_nm,img_nm in zip(csvs,tiles,images):
    # read csv file with fibroblast information
    csv = pd.read_csv(os.path.join(src,csv_nm))
    png = [os.path.join(src,_) for _ in os.listdir(os.path.join(src,img_nm)) if _.endswith('png')]
    png_positive = [int((os.path.splitext(os.path.split(_)[-1])[0])[5:]) for _ in png]
    positivity = pd.Series(np.zeros(len(csv)).astype(np.bool))
    positivity[png_positive]=True
    csv['isfibro']=positivity
    csv_nofb = csv.loc[csv['isfibro'] == False]
    if len(csv_nofb)<1: continue #there was no gating yet

    dst = os.path.join(src,tile_nm.replace('tile','false_positive'))
    if not os.path.exists(dst): os.mkdir(dst) # there was already filtering done for this
    else: continue
    csv.to_csv(os.path.join(src,csv_nm),index=False)
    dst_img = os.path.join(dst,'image')
    dst_mask = os.path.join(dst,'mask')
    if not os.path.exists(dst_img):os.mkdir(dst_img)
    if not os.path.exists(dst_mask):os.mkdir(dst_mask)

    img_name = csv_nm.replace('parameters','tissue_region').replace('csv','tif')
    img_pth = os.path.join(src,img_name)
    img = Image.open(img_pth)
    bw_img = Image.open(img_pth.replace('tissue_region','filtered'))
    xs = csv_nofb['x']
    ys = csv_nofb['y']
    for idx, row in csv_nofb.iterrows():
        x = row['x']
        y = row['y']
        (left, upper, right, lower) = (x - 50, y - 50, x + 50, y + 50)
        im_crop = img.crop((left, upper, right, lower))
        bw_crop = bw_img.crop((left, upper, right, lower))
        im_crop.save(os.path.join(dst_img,'tile_{:d}.png'.format(idx)))
        bw_crop.save(os.path.join(dst_mask,'tile_{:d}.png'.format(idx)))
